## JSON to MONGODB / SOLIDATA / CRUDITY

In [3]:
import os
import math
import pandas as pd
from dotenv import load_dotenv


In [6]:
### load variables from local .env file
load_dotenv()

True

In [8]:
### test loading env vars
print os.getenv("MYENVVAR")

secretenvvar
